# Emotiv BCI EEG Data Visualization with fastplotlib

Real-time rolling window visualization of EEG data with overlaid power spectra

In [19]:
# Import required libraries
import numpy as np
import pyedflib
import fastplotlib as fpl
import matplotlib.pyplot as plt

## Load EDF Data

In [20]:
# Read the EDF file
edf_file = 'data/EmotivBCI-BPM_EPOCX_357120_2025.10.05T17.46.53+02.00.edf'

# Open the file
f = pyedflib.EdfReader(edf_file)

# Get basic information
n_channels = f.signals_in_file
sample_frequencies = f.getSampleFrequencies()
channel_labels = f.getSignalLabels()

print(f"Number of channels: {n_channels}")
print(f"Sample frequency: {sample_frequencies[0]} Hz")
print(f"\nChannel labels:")
for i, label in enumerate(channel_labels[:20]):
    print(f"  {i}: {label}")

# EEG channels (common Emotiv electrodes)
eeg_channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 
                'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']

# Find indices of EEG channels
eeg_indices = []
for label in eeg_channels:
    if label in channel_labels:
        eeg_indices.append(channel_labels.index(label))

# Read all signals
sampling_rate = sample_frequencies[0] if len(sample_frequencies) > 0 else 128

all_signals = []
for ch_idx in eeg_indices:
    signal = f.readSignal(ch_idx)
    all_signals.append(signal)

f.close()

print(f"\nLoaded {len(all_signals)} EEG channels")
print(f"Signal length: {len(all_signals[0])} samples")

Number of channels: 60
Sample frequency: 128.0 Hz

Channel labels:
  0: TIME_STAMP_s
  1: TIME_STAMP_ms
  2: OR_TIME_STAMP_s
  3: OR_TIME_STAMP_ms
  4: COUNTER
  5: INTERPOLATED
  6: AF3
  7: F7
  8: F3
  9: FC5
  10: T7
  11: P7
  12: O1
  13: O2
  14: P8
  15: T8
  16: FC6
  17: F4
  18: F8
  19: AF4

Loaded 14 EEG channels
Signal length: 351104 samples


## Create fastplotlib Figure with Overlaid Subplots

Create a figure with multiple EEG trace subplots overlaid on a power spectrum background (matching matplotlib appearance)

In [21]:
# Rolling window parameters
window_size = 5000  # timepoints
step_size = 100  # datapoints per update

# Create extents for overlaid layout
# All EEG traces will be on left side, power spectrum on right
n_traces = len(eeg_indices)
trace_height = 1.0 / n_traces

# Create extents: EEG traces stacked vertically on left, power spectrum overlay on full left area
extents = []
names = []

# EEG trace subplots (stacked vertically)
for i in range(n_traces):
    y_min = i * trace_height
    y_max = (i + 1) * trace_height
    extents.append((0, 1, y_min, y_max))  # Full width for now
    names.append(f"trace_{i}")

# Create figure
figure = fpl.Figure(
    shape=(n_traces, 1),
    size=(1400, 900)
)

# Set black background
figure.canvas.set_logical_size(1400, 900)

# Store line graphics for each channel
eeg_lines = []
psd_lines = []

# Initialize with first window of data
current_pos = 0
x_data = np.arange(current_pos, current_pos + window_size)

# Create EEG trace lines (white)
for i, ch_idx in enumerate(eeg_indices):
    subplot = figure[i, 0]
    y_data = all_signals[i][current_pos:current_pos + window_size]
    
    # Add line for EEG trace
    line = subplot.add_line(
        np.column_stack([x_data, y_data]),
        thickness=2,
        colors="white",
        alpha=0.7
    )
    eeg_lines.append(line)
    
    # Customize subplot
    subplot.auto_scale()
    subplot.axes.visible = False
    
    # Add channel label as text
    subplot.title = channel_labels[ch_idx]
    subplot.title_color = "white"

print("Figure created with EEG traces")

RFBOutputContext()

Figure created with EEG traces


## Add Power Spectrum Overlay

Create a separate subplot for the power spectrum that overlays the EEG traces

In [22]:
# Add a separate figure for power spectra (we'll create a two-panel layout)
# Recreate figure with two columns: EEG traces (left) and power spectra (right)

figure = fpl.Figure(
    shape=(n_traces, 2),
    size=(1400, 900)
)

# Store line graphics
eeg_lines = []
psd_lines = []

# Initialize with first window
current_pos = 0
x_data = np.arange(current_pos, current_pos + window_size)

# Generate colors for each channel
colors = plt.cm.tab20(np.linspace(0, 1, n_traces))

# Left column: EEG traces (white on transparent)
for i, ch_idx in enumerate(eeg_indices):
    subplot = figure[i, 0]
    y_data = all_signals[i][current_pos:current_pos + window_size]
    
    line = subplot.add_line(
        np.column_stack([x_data, y_data]),
        thickness=2,
        colors="white",
        alpha=0.7
    )
    eeg_lines.append(line)
    
    subplot.auto_scale()
    subplot.axes.visible = False
    subplot.title = channel_labels[ch_idx]
    subplot.title_color = "white"
    subplot.camera.maintain_aspect = False

# Right column: Power spectra (log scale, colored lines)
for i in range(n_traces):
    subplot = figure[i, 1]
    
    # Compute initial power spectrum
    y_data = all_signals[i][current_pos:current_pos + window_size]
    freqs = np.fft.rfftfreq(window_size, 1/sampling_rate)
    pxx = np.abs(np.fft.rfft(y_data))**2
    
    # Add line with log scale
    line = subplot.add_line(
        np.column_stack([freqs, np.log10(pxx + 1e-10)]),
        thickness=2,
        colors=colors[i][:3],
        alpha=0.5
    )
    psd_lines.append(line)
    
    subplot.axes.visible = False
    subplot.camera.maintain_aspect = False

print("Figure created with EEG traces and power spectra")

RFBOutputContext()

Figure created with EEG traces and power spectra


## Setup Animation

Create animation function that updates the rolling window

In [23]:
# Animation state
max_samples = min(len(s) for s in all_signals)
current_pos = 0

def update_animation(subplot):
    """Update function called on each animation frame"""
    global current_pos
    
    # Restart from beginning if we've reached the end
    if current_pos + window_size > max_samples:
        current_pos = 0
    
    # New x data for rolling window
    x_data = np.arange(current_pos, current_pos + window_size)
    
    # Update EEG traces (left column)
    for i, line in enumerate(eeg_lines):
        y_data = all_signals[i][current_pos:current_pos + window_size]
        line.data[:, 0] = x_data
        line.data[:, 1] = y_data
    
    # Update power spectra (right column)
    for i, line in enumerate(psd_lines):
        y_data = all_signals[i][current_pos:current_pos + window_size]
        freqs = np.fft.rfftfreq(window_size, 1/sampling_rate)
        pxx = np.abs(np.fft.rfft(y_data))**2
        
        line.data[:, 0] = freqs
        line.data[:, 1] = np.log10(pxx + 1e-10)
    
    # Move window forward
    current_pos += step_size

# Add animation to first subplot (it will update all shared data)
figure[0, 0].add_animations(update_animation)

print("Animation setup complete")

Animation setup complete


## Display the Figure

Show the animated visualization (in JupyterLab, this will display inline)

In [24]:
# Show the figure using sidecar for side-by-side display
figure.show(sidecar=True)

print(f"Displaying rolling window: {window_size} timepoints")
print(f"Moving {step_size} points per frame")
print(f"Total samples: {max_samples}")

Displaying rolling window: 5000 timepoints
Moving 100 points per frame
Total samples: 351104
